In [1]:
import os
import getpass
from dotenv import load_dotenv
import langgraph

load_dotenv()

langchain_tracing = os.getenv("LANGCHAIN_TRACING_V2")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
aws_region = os.getenv("AWS_DEFAULT_REGION")
tavily_api_key = os.getenv("TAVILY_API_KEY")

Model

In [2]:
# 확인
from langchain_aws import ChatBedrock
from langchain_core.messages import HumanMessage

# Bedrock 모델 설정
model = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    model_kwargs=dict(temperature=0)
)

# 모델 호출 확인
example_result = model.invoke([HumanMessage(content="다음 미국 대선 후보는 누구인가요?")])
example_result

AIMessage(content='2024년 미국 대선의 주요 후보들은 아직 확정되지 않았습니다. 하지만 현재 상황을 보면:\n\n1. 민주당:\n   - 조 바이든 현 대통령이 재선에 도전할 가능성이 높습니다.\n\n2. 공화당:\n   - 도널드 트럼프 전 대통령이 출마를 선언했습니다.\n   - 론 드샌티스 플로리다 주지사가 유력한 후보로 거론되고 있습니다.\n   - 니키 헤일리 전 유엔 대사도 출마를 선언했습니다.\n\n그 외에도 여러 정치인들이 출마를 고려하거나 선언할 수 있습니다. 대선이 다가올수록 후보군이 더 명확해질 것입니다. 정확한 후보 명단은 각 당의 예비선거를 통해 결정될 것입니다.', additional_kwargs={'usage': {'prompt_tokens': 27, 'completion_tokens': 307, 'total_tokens': 334}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, response_metadata={'usage': {'prompt_tokens': 27, 'completion_tokens': 307, 'total_tokens': 334}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, id='run-5ad98a79-6743-4b2a-af4e-847675210d2a-0', usage_metadata={'input_tokens': 27, 'output_tokens': 307, 'total_tokens': 334})

Parser

In [3]:
from langchain_core.output_parsers import StrOutputParser

# 출력 파서 설정
parser = StrOutputParser()

# 파서 확인
parsed_result = parser.invoke(example_result)
print(parsed_result)  # 출력: '안녕!'

2024년 미국 대선의 주요 후보들은 아직 확정되지 않았습니다. 하지만 현재 상황을 보면:

1. 민주당:
   - 조 바이든 현 대통령이 재선에 도전할 가능성이 높습니다.

2. 공화당:
   - 도널드 트럼프 전 대통령이 출마를 선언했습니다.
   - 론 드샌티스 플로리다 주지사가 유력한 후보로 거론되고 있습니다.
   - 니키 헤일리 전 유엔 대사도 출마를 선언했습니다.

그 외에도 여러 정치인들이 출마를 고려하거나 선언할 수 있습니다. 대선이 다가올수록 후보군이 더 명확해질 것입니다. 정확한 후보 명단은 각 당의 예비선거를 통해 결정될 것입니다.


Prompt Template

In [7]:
from langchain_core.prompts import ChatPromptTemplate

# 시스템 템플릿
system_template = (
    "당신은 질문에 답하는 어시스턴트입니다. "
    "다음 문서에서 제공된 내용을 사용하여 질문에 답변하세요. "
    "모르면 모른다고 말하세요. 최대 세 문장으로 간결하게 답하세요."
    "\\n\\n"
    "{context}"
)

# 프롬프트 템플릿
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template), 
        ("user", "{input}"),
    ]
)

# # 템플릿 호출
# prompt_result = prompt_template.invoke()
# print(prompt_result.to_messages())

TypeError: BasePromptTemplate.invoke() missing 1 required positional argument: 'input'

메모리(+체인)

In [ ]:
from langchain_aws import BedrockEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 스플리터 설정 (1000자 단위로 문서를 나누고, 중첩 200자 적용)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# 벡터 스토어에 임베딩 적용 및 문서 저장
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=BedrockEmbeddings(
        model_id='amazon.titan-embed-text-v1',
    )
)

# 리트리버 생성
retriever = vectorstore.as_retriever()

In [6]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# 새로운 그래프 정의
workflow = StateGraph(state_schema=MessagesState)

# # 호출 함수
# def call_model(state: MessagesState):
#     # 체인 생성
#     chain = prompt_template | model | parser
#     response = chain.invoke(state["messages"])
#     return {"messages": response}

# # 노드 및 메모리 설정(메모리와 컴파일)
# workflow.add_edge(START, "model")
# workflow.add_node("model", call_model)
# memory = MemorySaver()
# app = workflow.compile(checkpointer=memory)

# def create_qna_chain(model, prompt_template, parser):
#     return prompt_template | model | parser

qna_chain = create_stuff_documents_chain(model, prompt_template, parser)

rag_chain = create_retrieval_chain(retriever, qna_chain)